In [1]:
!wget https://raw.githubusercontent.com/surya1203/Toxic-Spans-Detection/testing/crf_helper.py

--2022-12-22 09:33:59--  https://raw.githubusercontent.com/surya1203/Toxic-Spans-Detection/testing/crf_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6660 (6.5K) [text/plain]
Saving to: ‘crf_helper.py’

crf_helper.py       100%[===================>]   6.50K  --.-KB/s    in 0s      

2022-12-22 09:33:59 (85.8 MB/s) - ‘crf_helper.py’ saved [6660/6660]



In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.9 MB/s 
     |████████████████████████████████| 1.1 MB 60.4 MB/s 
     |████████████████████████████████| 7.6 MB 69.3 MB/s 
     |████████████████████████████████| 182 kB 81.4 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=208aac62699f1b3acfabfe80fb1ca4ee9081c4cdc7878611f105c2b1ab58804a
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


## Creating Directories if needed

In [3]:
root = './'

In [4]:
import os
def create_directories(root_path):
  checkpoints_dir = os.path.join(root_path,'T2_Checkpoints_Giri_Surya')
  # lstm_dir = os.path.join(root_path, 'validation')

  os.makedirs(checkpoints_dir + '/lstm')
  os.makedirs(checkpoints_dir + '/bilstm')

create_directories(root)

## Copy data to directories (not working as files are too big)

In [30]:
import requests
def copy_weights_bilstm(path):
  # save_path = 
  print(is_downloadable('https://drive.google.com/file/d/1-lSL6erMZOqXARW47xRB3wc_vJj1-a-g/view?usp=share_link'))
  open(os.path.join(path,'bilstm.data-00000-of-00001'), 'wb').write(requests.get('https://drive.google.com/file/d/1-lSL6erMZOqXARW47xRB3wc_vJj1-a-g/view?usp=share_link', allow_redirects=True).content)
  open(os.path.join(path,'checkpoint'), 'wb').write(requests.get('https://drive.google.com/file/d/1-zLjeuNIAHnqRWxlKlTf-BILbxSns--j/view?usp=sharing', allow_redirects=True).content)
  open(os.path.join(path,'bilstm.index'), 'wb').write(requests.get('https://drive.google.com/file/d/107DaY82DIKe121NJQRQM3lpLQvfARfOW/view?usp=share_link', allow_redirects=True).content)

def copy_weights_lstm(path):
  open(os.path.join(path,'lstm.data-00000-of-00001'), 'wb').write(requests.get('https://drive.google.com/file/d/1-6OdmsIv3LvVKMV9fQeQUkb2HBI2rlEW/view?usp=share_link', allow_redirects=True).content)
  open(os.path.join(path,'checkpoint'), 'wb').write(requests.get('https://drive.google.com/file/d/1-EL_AW0IqI1vzHZEwwjt3FmgyQHCAnOO/view?usp=sharing', allow_redirects=True).content)
  open(os.path.join(path,'lstm.index'), 'wb').write(requests.get('https://drive.google.com/file/d/1-GAmbe0ZMtfxf7I3jhZ09U_i-c6iaIdE/view?usp=share_link', allow_redirects=True).content)


copy_weights_bilstm('/content/T2_Checkpoints_Giri_Surya/bilstm')
copy_weights_lstm('/content/T2_Checkpoints_Giri_Surya/lstm')

NameError: ignored

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

## Loading Libraries

In [7]:
import pandas as pd
import numpy as np
import json
import gc
# import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from transformers import TFElectraModel,ElectraTokenizer
from keras import callbacks
import os

## Load dataset


In [8]:
# Training
train_dataset = pd.read_csv('tsd_train.csv')
test_dataset = pd.read_csv('tsd_test.csv')

In [9]:
# Creating python dictionary from json string for spans
train_dataset['spans'] = train_dataset['spans'].apply(lambda x : json.loads(x))
test_dataset['spans'] = test_dataset['spans'].apply(lambda x : json.loads(x))

In [10]:
# Converting spans to numpy
spans = train_dataset['spans'].to_numpy()
# spans

In [11]:
# outputs = create_outputs(texts,train_dataset['spans'],max_length,tokenizer)
# kf = KFold(n_splits=5)
# train_validation_indices = []
# for train_index,validation_index in kf.split(texts):
#     train_validation_indices.append((train_index,validation_index))

## Tokenize Using Electra

In [12]:
# Tokenizer
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

## Generate inputs and outputs

In [13]:
max_length = 400
def create_outputs(texts,spans,max_length,tokenizer):
    outputs = []
    for text,span in zip(texts,spans):
        # initialize outputs with 0
        output = np.zeros(max_length*3,dtype=np.float).reshape((max_length,3))
        tokens = tokenizer.tokenize(text)[:max_length]
        length = 0
        start = True
        for i in range(len(tokens),max_length):
            output[i,0] = 1.0
        for index,token in enumerate(tokens):
            sub = False
            if "##" in token:
                sub = True
                token = token[2:]
            if not start:
                next_index = text[length:].find(token)
                if next_index == 0:
                    sub = True
                length += next_index
            # if length in span and not sub:
            #     output[index,2] = 1.0
            #     output[index,0] = 0.0
            if length in span:
                output[index,2] = 1.0
                output[index,0] = 0.0
            else:
                output[index,1] = 1.0
                output[index,0] = 0.0
            length += len(token)
            start = False
        outputs.append(output)
    return np.array(outputs)

In [14]:
def create_inputs(texts,max_length,tokenizer):
  # Tokenize the input texts
  tokens = tokenizer(texts, max_length=max_length, padding="max_length", return_tensors="tf",truncation=True)
  # Get input length for each text
  input_length = []
  for text in texts:
    input_length.append(min(max_length,len(tokenizer.tokenize(text))))

  # Generate arrays of the tokenized inputs
  # Input IDs
  tokenized_input_ids = np.array(tokens['input_ids'])
  # Attention mask
  tokenized_token_type_ids = np.array(tokens['token_type_ids'])
  # Token type IDs
  tokenized_attention_mask = np.array(tokens['attention_mask'])
  # Input length
  nparray_input_length = np.array(input_length)
  
  inputs = [tokenized_input_ids, tokenized_token_type_ids, tokenized_attention_mask, nparray_input_length]
  return inputs

In [15]:
# Converting texts to numpy
texts = train_dataset['text'].to_numpy()
# texts

## Split training and testing datasets into x and y

In [16]:
# # train_index, validation_index = train_validation_indices.pop()
# # y_train, y_validation = outputs[train_index], outputs[validation_index]
# train_index, validation_index = train_validation_indices.pop()

# Get texts for train and test set
x_train, x_test= list(train_dataset['text'].to_numpy()), list(test_dataset['text'].to_numpy())

# Get spans for train and test set
y_train = create_outputs(train_dataset['text'].to_numpy(),train_dataset['spans'],max_length,tokenizer)
y_test = create_outputs(test_dataset['text'].to_numpy(),test_dataset['spans'],max_length,tokenizer)

<ipython-input-13-c594bca6b129>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  output = np.zeros(max_length*3,dtype=np.float).reshape((max_length,3))


## Build Model

In [17]:
from crf_helper import *

In [18]:
def build_lstm_model(max_input_length,base_model):
    input_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_input_ids",dtype=tf.int32)
    input_type_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_token_type_ids",dtype=tf.int32)
    input_attention_mask_layer = layers.Input(shape=(max_input_length,),name="encoder_attention_mask",dtype=tf.int32)
    input_length = layers.Input(shape=(1,),name="length",dtype=tf.int32)
    base_model.trainable = True
    # a list of varying length with one or several input Tensors IN THE ORDER given in the docstring: 
    # model([input_ids, attention_mask]) or model([input_ids, attention_mask, token_type_ids])
    base_model = base_model(input_ids_layer,token_type_ids=input_type_ids_layer,attention_mask=input_attention_mask_layer,return_dict=True)
    output = layers.LSTM(512,return_sequences=True)(base_model.last_hidden_state)
    output = layers.Dense(3,activation="linear")(output)
    crf = CRFLayer()
    output = crf(inputs=[output,input_length])
    model = models.Model(inputs=[input_ids_layer,input_type_ids_layer,input_attention_mask_layer,input_length],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=3e-5),loss=crf.loss,metrics=['accuracy'])
    return model

In [19]:
def build_bilstm_model(max_input_length,base_model):
    input_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_input_ids",dtype=tf.int32)
    input_type_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_token_type_ids",dtype=tf.int32)
    input_attention_mask_layer = layers.Input(shape=(max_input_length,),name="encoder_attention_mask",dtype=tf.int32)
    input_length = layers.Input(shape=(1,),name="length",dtype=tf.int32)
    base_model.trainable = True
    # a list of varying length with one or several input Tensors IN THE ORDER given in the docstring: 
    # model([input_ids, attention_mask]) or model([input_ids, attention_mask, token_type_ids])
    base_model = base_model(input_ids_layer,token_type_ids=input_type_ids_layer,attention_mask=input_attention_mask_layer,return_dict=True)
    output = layers.Bidirectional(layers.LSTM(512,return_sequences=True))(base_model.last_hidden_state)
    output = layers.Dense(3,activation="linear")(output)
    crf = CRFLayer()
    output = crf(inputs=[output,input_length])
    model = models.Model(inputs=[input_ids_layer,input_type_ids_layer,input_attention_mask_layer,input_length],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=3e-5),loss=crf.loss,metrics=['accuracy'])
    return model

## Fit LSTM model

In [20]:
gc.collect()
tf.keras.backend.clear_session()
# Base model Electra
base_model = TFElectraModel.from_pretrained('google/electra-base-discriminator')

# LSTM Model
model_lstm = build_lstm_model(max_length,base_model)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-base-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [21]:
# tokenize texts for training
train_data = create_inputs(x_train,max_length,tokenizer)

# tokenize texts for testing
test_data = create_inputs(x_test,max_length,tokenizer)

In [22]:
# Get spans for testing
spans_test = test_dataset['spans']

# Fet spans for training
spans_train = train_dataset['spans']

In [23]:
def NERGetIndicesSingleText(outputs,text,tokenizer):
    outputs = tf.argmax(outputs,axis=-1)
    tokens = tokenizer.tokenize(text)
    index = 0
    indexes = []
    sub = False
    prev = False
    for token,output in zip(tokens,outputs):
      # end token for pretrained embeddings from Electra
        if token[:2] == "##":
            token = token[2:]
            sub = True
        else:
            sub = False
        temp_index = text[index:].find(token)
        temp_start = index+temp_index
        if output == 2 or (sub and prev and output != 0):
            prev = True
            indexes = indexes + list(range(temp_start,temp_start+len(token)))
        else:
            prev = False
        index = temp_start+len(token)
    return np.array(indexes)

In [24]:
def createIndicesForNERModel(predicts,texts,tokenizer):
    outputs = []
    for text,pred in zip(texts,predicts):
         indices = NERGetIndicesSingleText(pred,text,tokenizer)
         outputs.append(indices)
    return outputs

In [25]:
def f1(preds,trues):
    
    if len(trues) == 0:
        # return 1 if true is empty
        return 1. if len(preds) == 0 else 0.
    if len(preds) == 0:
        # return 0 if preds is empty
        return 0.
    predictions_set = set(preds)
    gold_set = set(trues)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [26]:
def avg_f1(preds,trues):
  # Initialize average f1
    average_f1_total = 0.0
    for pred,true in zip(preds,trues):
      # Calculate average of f1 scores
        average_f1_total += f1(pred,true)
    return average_f1_total/len(preds)

In [27]:
class F1Metric(callbacks.Callback):
    def __init__(self,inputs,labels,spans,texts,test=True):
        self.inputs = inputs
        self.spans = spans
        self.tokenizer = tokenizer
        self.texts = texts
        self.test = test

    def on_epoch_end(self, epoch, logs={}):
        preds = self.model.predict(self.inputs,verbose=0)
        indices = createIndicesForNERModel(preds,texts,tokenizer)
        f1 = avg_f1(indices,self.spans)
        if self.test:
            print()
            print("test f1 = "+str(f1))
        else:
            print()
            print("train f1 = "+str(f1))

In [33]:
# model_lstm.fit(train_data,y_train,batch_size=16,epochs=2,callbacks=[callbacks.ModelCheckpoint("/content/drive/MyDrive/T2_Checkpoints_Giri_Surya/lstm/checkpoints/lstm",save_weights_only=True)])

# Load weights
model_lstm.load_weights('/content/drive/MyDrive/T2_Checkpoints_Giri_Surya/lstm/checkpoints/lstm')
# model_lstm.load_weights('./T2_Checkpoints_Giri_Surya/lstm/')

In [34]:
# Get predictions
preds_lstm = model_lstm.predict(test_data)

# Generate indices of the toxic spans
indices = createIndicesForNERModel(preds_lstm,x_test,tokenizer)

# Calculate F1 score of the prediction
f1_toxic = avg_f1(indices,spans_test)

63/63 [==============================] - 74s 1s/step


In [35]:
print("test F1 = %f"%(f1_toxic))

test F1 = 0.473530


## Bidirectional LSTM model

In [36]:
gc.collect()
tf.keras.backend.clear_session()

# Base model Electra
base_model = TFElectraModel.from_pretrained('google/electra-base-discriminator')

# Bidirectional LSTM Model
model_bilstm = build_bilstm_model(max_length,base_model)

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-base-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.


In [37]:
# model_bilstm.fit(train_data,y_train,batch_size=8,epochs=2,callbacks=[callbacks.ModelCheckpoint("/content/drive/MyDrive/T2_Checkpoints_Giri_Surya/bilstm/checkpoints/bilstm",save_weights_only=True)])

# Load the weights saved during training
model_bilstm.load_weights('/content/drive/MyDrive/T2_Checkpoints_Giri_Surya/bilstm/checkpoints/bilstm')
# model_bilstm.load_weights('./T2_Checkpoints_Giri_Surya/bilstm')

# Get predictions
preds_bilstm = model_bilstm.predict(test_data)

# Generate indices for the toxic spans
indices = createIndicesForNERModel(preds_bilstm,x_test,tokenizer)

# Calculate f1 score
f1_toxic = avg_f1(indices,spans_test)
print("test F1 = %f"%(f1_toxic))

63/63 [==============================] - 76s 1s/step
test F1 = 0.477233
